In [1]:
import sqlite3
import pandas as pd
import numpy as np
df = pd.DataFrame({"name":["a","b","c","a","a", "a"],
                  "cost":[-2, 5, 2, 2, 7, 2],
                 "sex":["F","M","M","F","F", "F"]})
df

,name,cost,sex
0,a,-2,F
1,b,5,M
2,c,2,M
3,a,2,F
4,a,7,F
5,a,2,F


In [2]:
import sqlite3
conn = sqlite3.connect(':memory:')
df.to_sql('table1', conn, index=False)

qry1 = '''
    select tab1.name,tab1.cost,tab1.sex
    from table1 tab1 left join table1 tab2 
    on tab1.name =tab2.name and tab1.cost = -tab2.cost
    where tab2.name is null
    '''
df_keep = pd.read_sql_query(qry1, conn)
df_keep.to_sql('table2', conn, index=False)
df_keep

,name,cost,sex
0,b,5,M
1,c,2,M
2,a,7,F


In [3]:
qry2 = '''
    SELECT * FROM table1
    WHERE (name, cost) not in 
    (SELECT name, cost FROM table2)
    '''
df_d = pd.read_sql_query(qry2, conn)
df_d.to_sql('table3', conn, index=False)
df_d

,name,cost,sex
0,a,-2,F
1,a,2,F
2,a,2,F


In [4]:
qry3 = '''
SELECT * FROM table3
where cost > 0
group by name, cost
having count(*) > 1
'''
df_retain = pd.read_sql_query(qry3, conn)
df_retain.to_sql('table4', conn, index=False)
df_retain

,name,cost,sex
0,a,2,F


In [5]:
qry4 = '''
    SELECT * FROM table2
    UNION
    SELECT * FROM table4
    '''
pd.read_sql_query(qry4, conn)

,name,cost,sex
0,a,2,F
1,a,7,F
2,b,5,M
3,c,2,M
